In [176]:
import pandas as pd
import numpy as np
from pulp import *
import re
from sklearn.ensemble import RandomForestRegressor
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split

In [2]:
allseasons = pd.read_csv('/Users/jbachlombardo/Documents/Python/Projects/xG/all_seasons_leagues_1415-1819.csv')

In [5]:
allseasons = allseasons.drop('Unnamed: 0', axis = 1)

In [6]:
costs = pd.read_csv('/Users/jbachlombardo/Documents/Python/Projects/xG/fpl_costs_120819.csv')

In [7]:
points = pd.read_csv('/Users/jbachlombardo/Documents/Python/Projects/xG/fpl_points1718_120819.csv')

In [11]:
costs = costs.dropna(axis = 1)
points = points.dropna(axis = 1)

In [15]:
costs = costs.rename(columns = {'Name': 'Player'})

In [26]:
costspoints = costs.merge(points, on = ['Player', 'Team', 'Position'])

In [27]:
team_dict = {
'ARS': 'Arsenal',
'AVL': 'Aston Villa',
'BHA': 'Brighton Hove Albion',
'BOU': 'Bournemouth',
'BUR': 'Burnley',
'CHE': 'Chelsea',
'CRY': 'Crystal Palace',
'EVE': 'Everton',
'LEI': 'Leicester',
'LIV': 'Liverpool',
'MCI': 'Manchester City',
'MUN': 'Manchester United',
'NEW': 'Newcastle United',
'NOR': 'Norwich',
'SHU': 'Sheffield United',
'SOU': 'Southampton',
'TOT': 'Tottenham',
'WAT': 'Watford',
'WHU': 'West Ham',
'WOL': 'Wolverhampton Wanderers'
}

In [29]:
costspoints['Team_name'] = costspoints['Team'].map(team_dict)

In [30]:
def get_position_cost(x, y) :
    last = x.split()[-1]
    results = costspoints.loc[(costspoints['Player'].str.contains(last)) & (costspoints['Team_name'] == y), ['Cost', 'Position']].values
    try :
        return list(results[0])
    except :
        return [np.nan, np.nan]

In [31]:
def get_position_points(x, y) :
    last = x.split()[-1]
    results = costspoints.loc[(costspoints['Player'].str.contains(last)) & (costspoints['Team_name'] == y), ['Points', 'Position']].values
    try :
        return list(results[0])
    except :
        return [np.nan, np.nan]

In [32]:
season2018 = allseasons[(allseasons['League'] == 'EPL') & (allseasons['Season'] == '2017/18')]

In [34]:
season2018_results = season2018[['Player', 'Team']].apply(lambda x: get_position_points(x[0], x[1]), axis = 1)

In [35]:
season2018[['FPL_Points', 'FPL_Pos']] = pd.DataFrame(season2018_results.values.tolist(), index = season2018_results.index)

/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [38]:
season2018 = season2018.dropna()

In [45]:
lm = smf.ols('FPL_Points ~ xG + xA + xG90 + xA90', data = season2018).fit()

In [194]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             FPL_Points   R-squared:                       0.611
Model:                            OLS   Adj. R-squared:                  0.604
Method:                 Least Squares   F-statistic:                     87.22
Date:                Sat, 17 Aug 2019   Prob (F-statistic):           2.02e-44
Time:                        10:57:10   Log-Likelihood:                -1099.8
No. Observations:                 227   AIC:                             2210.
Df Residuals:                     222   BIC:                             2227.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     49.9609      2.844     17.570      0.000      44.357      55.565
xG             4.9562      0.661      7.502      0.000       3.654       6.258
xA            12.9914      1.384      9.386      0.000      10.264      15.719
xG90         -11.5075      4.368     -2.634      0.009     -20.116      -2.899
xA90         -80.7760     23.116     -3.494      0.001    -126.330     -35.221
==============================================================================
Omnibus:                       28.841   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               36.941
Skew:                           0.854   Prob(JB):                     9.52e-09
Kurtosis:                       3.994   Cond. No.                         59.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [47]:
X = season2018[['xG', 'xA', 'xG90', 'xA90']].values
y = season2018['FPL_Points'].values

In [48]:
reg = RandomForestRegressor()

In [49]:
reg = reg.fit(X, y)

/anaconda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [52]:
reg.feature_importances_

array([0.29999028, 0.53299538, 0.06209455, 0.10491979])

In [55]:
reg.score(X, y)

0.9036032063288963

In [192]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [193]:
reg_test = RandomForestRegressor()
reg_test = reg_test.fit(X_train, y_train)
reg_test.score(X_test, y_test)

/anaconda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5389678196960636

In [62]:
allseasons_costs = pd.read_csv('/Users/jbachlombardo/Documents/Python/Projects/xG/all_seasons_leagues_1415-1819_withcosts.csv')

In [64]:
allseasons_costs = allseasons_costs.drop('Unnamed: 0', axis = 1)

In [67]:
with_costs = allseasons_costs.dropna(subset = ['FPL_Cost'])

In [69]:
with_costs['Sh'] = with_costs['Sh90'] * with_costs['Min'] / 90
with_costs['KP'] = with_costs['KP90'] * with_costs['Min'] / 90

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [70]:
agg_dict = {
    'xG': 'sum',
    'xA': 'sum',
    'G': 'sum',
    'A': 'sum',
    'Min': 'sum',
    'Sh': 'sum',
    'KP': 'sum',
    'Season': 'count',
    'FPL_Cost': 'max'
}

In [130]:
players_totals = with_costs.groupby(['Player', 'Team', 'FPL_Pos'], as_index = False).agg(agg_dict)

In [131]:
players_three = with_costs[(with_costs['Season'] == '2016/17') | (with_costs['Season'] == '2017/18') | (with_costs['Season'] == '2018/19')]

In [132]:
players_two = with_costs[(with_costs['Season'] == '2017/18') | (with_costs['Season'] == '2018/19')]

In [133]:
players_one = with_costs[with_costs['Season'] == '2018/19']

In [134]:
players_three = players_three.groupby(['Player', 'Team', 'FPL_Pos'], as_index = False).agg(agg_dict)

In [135]:
players_two = players_two.groupby(['Player', 'Team', 'FPL_Pos'], as_index = False).agg(agg_dict)

In [136]:
rate_cols = ['xG', 'xA', 'G', 'A', 'Sh', 'KP']
for r in rate_cols :
    new_col = r + '90'
    players_totals[new_col] = players_totals[r] / players_totals['Min'] * 90

In [137]:
for r in rate_cols :
    new_col = r + '90'
    players_three[new_col] = players_three[r] / players_three['Min'] * 90

In [138]:
for r in rate_cols :
    new_col = r + '90'
    players_two[new_col] = players_two[r] / players_two['Min'] * 90

In [139]:
players_totals = players_totals.rename(columns = {'xG': 'xG_total', 'xA': 'xA_total'})
players_three = players_three.rename(columns = {'xG': 'xG_total', 'xA': 'xA_total'})
players_two = players_two.rename(columns = {'xG': 'xG_total', 'xA': 'xA_total'})

In [140]:
players_totals['xG'] = players_totals['xG_total'] / players_totals['Season']
players_three['xG'] = players_three['xG_total'] / players_three['Season']
players_two['xG'] = players_two['xG_total'] / players_two['Season']

In [141]:
players_totals['xA'] = players_totals['xA_total'] / players_totals['Season']
players_three['xA'] = players_three['xA_total'] / players_three['Season']
players_two['xA'] = players_two['xA_total'] / players_two['Season']

In [142]:
defs = season2018[season2018['FPL_Pos'] == 'DEF']
mids = season2018[season2018['FPL_Pos'] == 'MID']
fwds = season2018[season2018['FPL_Pos'] == 'FWD']

In [143]:
X_defs = defs[['xG', 'xA', 'xG90', 'xA90']]
y_defs = defs['FPL_Points']
reg_defs = RandomForestRegressor()
reg_defs = reg_defs.fit(X_defs, y_defs)

/anaconda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [144]:
X_mids = mids[['xG', 'xA', 'xG90', 'xA90']]
y_mids = mids['FPL_Points']
reg_mids = RandomForestRegressor()
reg_mids = reg_mids.fit(X_mids, y_mids)

/anaconda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [145]:
X_fwds = fwds[['xG', 'xA', 'xG90', 'xA90']]
y_fwds = fwds['FPL_Points']
reg_fwds = RandomForestRegressor()
reg_fwds = reg_fwds.fit(X_fwds, y_fwds)

/anaconda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [146]:
def pred_scores(data) :
    defs_pred = data[data['FPL_Pos'] == 'DEF']
    mids_pred = data[data['FPL_Pos'] == 'MID']
    fwds_pred = data[data['FPL_Pos'] == 'FWD']
    defs_pred['Pred_score'] = reg_defs.predict(defs_pred[['xG', 'xA', 'xG90', 'xA90']])
    mids_pred['Pred_score'] = reg_mids.predict(mids_pred[['xG', 'xA', 'xG90', 'xA90']])
    fwds_pred['Pred_score'] = reg_defs.predict(fwds_pred[['xG', 'xA', 'xG90', 'xA90']])
    all_predicted = pd.concat([defs_pred, mids_pred, fwds_pred])
    return all_predicted

In [149]:
players_totals = pred_scores(players_totals)
players_three = pred_scores(players_three)
players_two = pred_scores(players_two)
players_one = pred_scores(players_one)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

In [151]:
index_cols = ['Player', 'Team', 'FPL_Pos', 'FPL_Cost']

In [152]:
players_totals = players_totals.set_index(index_cols)
players_three = players_three.set_index(index_cols)
players_two = players_two.set_index(index_cols)
players_one = players_one.set_index(index_cols)

In [153]:
player_scores = pd.DataFrame(index = players_totals.index)

In [154]:
player_scores['Score_5'] = players_totals['Pred_score'] * 0.05
player_scores['Score_3'] = players_three['Pred_score'] * 0.15
player_scores['Score_2'] = players_two['Pred_score'] * 0.30
player_scores['Score_1'] = players_one['Pred_score'] * 0.50

In [156]:
player_scores['Score_Total'] = player_scores.sum(axis = 1)

In [157]:
player_scores = player_scores.reset_index()

In [158]:
players = player_scores['Player'].values
fwds = player_scores[player_scores['FPL_Pos'] == 'FWD']['Player'].values
mids = player_scores[player_scores['FPL_Pos'] == 'MID']['Player'].values
defs = player_scores[player_scores['FPL_Pos'] == 'DEF']['Player'].values

In [160]:
player_points = player_scores.set_index('Player')['Score_Total'].to_dict()
player_costs = player_scores.set_index('Player')['FPL_Cost'].to_dict()

In [161]:
fantasy = LpProblem('max_score_fantasy', LpMaximize)

In [162]:
player_vars = LpVariable.dicts('Player_', players, cat = 'Binary')

In [163]:
fantasy += lpSum([player_points[i] * player_vars[i] for i in players]), 'max_points'

In [164]:
fantasy += lpSum([player_costs[p]*player_vars[p] for p in players]) <= 90, 'spending_limit'
fantasy += lpSum([player_vars[p] for p in players if p in fwds]) == 3, 'forwards'
fantasy += lpSum([player_vars[p] for p in players if p in mids]) == 5, 'midfielders'
fantasy += lpSum([player_vars[p] for p in players if p in defs]) == 5, 'defenders'

In [165]:
fantasy.solve()

1

In [166]:
LpStatus[fantasy.status]

'Optimal'

In [167]:
in_team = {}
for v in fantasy.variables() :
    name = v.name[8:].replace('_', ' ')
    in_team[name] = v.varValue

In [168]:
player_scores['In_team'] = player_scores['Player'].map(in_team)

In [169]:
player_scores[player_scores['In_team'] == 1]

,Player,Team,FPL_Pos,FPL_Cost,Score_5,Score_3,Score_2,Score_1,Score_Total,In_team
67,José Holebas,Watford,DEF,5.0,5.235,22.065,38.67,65.55,131.520,1.0
84,Marcos Alonso,Chelsea,DEF,6.5,7.275,21.825,43.65,72.25,145.000,1.0
101,Patrick van Aanholt,Crystal Palace,DEF,5.5,4.220,12.660,41.70,78.80,137.380,1.0
105,Ricardo Pereira,Leicester,DEF,6.0,7.135,21.405,42.81,69.25,140.600,1.0
126,Willy Boly,Wolverhampton Wanderers,DEF,5.0,6.800,20.400,40.80,68.00,136.000,1.0
155,Christian Eriksen,Tottenham,MID,9.0,9.090,29.550,55.44,75.65,169.730,1.0
158,David Brooks,Bournemouth,MID,6.5,7.165,21.495,42.99,71.65,143.300,1.0
159,David Silva,Manchester City,MID,8.0,7.545,25.905,49.80,86.35,169.600,1.0
193,James Maddison,Leicester,MID,7.0,7.795,23.385,46.77,77.95,155.900,1.0
236,Mohamed Salah,Liverpool,MID,12.5,13.670,41.010,82.02,125.60,262.300,1.0
